In [ ]:
# detection
# https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
# https://chatgpt.com/c/67fa0c48-7acc-8003-bb1b-966899fcc1a4

import os
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import backbones
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from torchvision.transforms import functional as F


# Подготовка кастомного датасета
class CustomObjectDetectionDataset(Dataset):
    def __init__(self, image_dir, annotations, transform=None):
        self.image_dir = image_dir
        self.annotations = annotations
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Загружаем изображение
        img_name = os.path.join(self.image_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")

        # Загружаем bounding box и классы
        boxes = (
            self.annotations.iloc[idx, 1:5].values.astype(float).reshape(-1, 4)
        )  # координаты bbox
        labels = torch.tensor([self.annotations.iloc[idx, 5]])  # метка класса

        # Преобразования (если необходимо)
        if self.transform:
            image = self.transform(image)

        # Преобразование в формат для PyTorch
        target = {"boxes": boxes, "labels": labels}

        return image, target


# Загрузим аннотации и изображения
# Предположим, что файл CSV имеет такой формат: 'image_name', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id'
annotations = pd.read_csv("annotations.csv")

# Создадим преобразования
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

# Путь к папке с изображениями
image_dir = "path_to_images/"

# Создание датасета
dataset = CustomObjectDetectionDataset(image_dir, annotations, transform)

# Создание DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Загружаем предобученную модель Faster R-CNN с backbone ResNet
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Настройка на одну задачу
num_classes = 2  # Включая класс фона
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = (
    torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
)

# Модель в режим тренировки
model.train()

# Используем оптимизатор
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Цикл тренировки
num_epochs = 10

for epoch in range(num_epochs):
    for images, targets in dataloader:
        # Перемещаем данные на GPU, если оно доступно
        images = [image.to("cuda") for image in images]
        targets = [{k: v.to("cuda") for k, v in t.items()} for t in targets]

        # Обучение модели
        optimizer.zero_grad()
        loss_dict = model(images, targets)

        # Потери
        losses = sum(loss for loss in loss_dict.values())

        # Обратный проход
        losses.backward()
        optimizer.step()

    print(f"Epoch #{epoch} Loss: {losses.item()}")

# Сохраним модель
torch.save(model.state_dict(), "fasterrcnn.pth")